## Step 2 - Climate App

 Design a Flask API based on the queries saved in 'climate_starter.ipynb'
 
* Routes
       * `/`
           * Home page.
           * List all routes that are available.
       * `/api/v1.0/precipitation`
           * Convert the query results to a Dictionary using `date` as the key and `prcp` as the value.
           * Return the JSON representation of your dictionary.
       * `/api/v1.0/stations`
           * Return a JSON list of stations from the dataset.
       * `/api/v1.0/tobs`
           * Query the dates and temperature observations of the most active station for the last year of data.
           * Return a JSON list of Temperature Observations (tobs) for the previous year
       * `/api/v1.0/<start>` and `/api/v1.0/<start>/<end>`
           * Return a JSON list of the minimum temperature, the average temperature, and
               the max temperature for a given start or start-end range.
           * When given the start only, calculate `TMIN`, `TAVG`, and `TMAX` for all dates 
               greater than and equal to the start date.
           * When given the start and the end date, calculate the `TMIN`, `TAVG`, and `TMAX` 
               for dates between the start and end date inclusive.

In [1]:
# Import Dependencies 
import pandas as pd
import numpy as np

import datetime as dt

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

from flask import Flask, jsonify

In [2]:
# Database setup
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

# Reflect an existing database into a new model
Base = automap_base()

# Reflect the tables
Base.prepare(engine, reflect=True)

# Save reference to the tables
Measurement = Base.classes.measurement
Station = Base.classes.station

# Create our session (link) from Python to the DB
session = Session(engine)

# Flask Setup:
app = Flask(__name__)

In [3]:
# Home page.
# List all routes that are available.

@app.route("/")
def home_page():
    return (
        f"<div align='center'><h1>Climate App</h1><br/></div>"
        f"<p>Usage:</p>"
        f"/api/v1.0/precipitation<br/>Precipitation data for the dates between 8/23/16 and 8/23/17<br/><br/>"
        f"/api/v1.0/stations<br/>Station numbers and names:<br/><br/>"
        f"/api/v1.0/tobs<br/>Temperature Observations for each station for the dates between 8/23/16 and 8/23/17<br/><br/>"
        f"/api/v1.0/date<br/>Minimum temperature, the average temperature, and the max temperature for the dates between the given start date and 8/23/17<br/><br/>."
        f"/api/v1.0/start_date/end_date<br/>Minimum temperature, the average temperature, and the max temperature for the dates between the given start date and end date<br/><br/>."
    )

In [4]:
# Convert the query results to a dictionary using date as the key and prcp as the value.
# Return the JSON representation of your dictionary.

@app.route("/api/v1.0/precipitation")  
def precipitation():
    last_yr = dt.date(2017,8,23) - dt.timedelta(days = 365)
    last_day = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
    precipitation = session.query(Measurement.date, Measurement.prcp).\
        filter(Measurement.date > last_yr).order_by(Measurement.date).all()

    precipitation_data = []
    for i in precipitation:
        data = {}
        data['date'] = precipitation[0]
        data['prcp'] = precipitation[1]
        precipitation_data.append(data)
    return jsonify(precipitation_data)

In [5]:
## Return a JSON list of stations from the dataset.
@app.route("/api/v1.0/stations")
    station_results = session.query(Stations.station).all()
    station_all = list(np.ravel(results))
    return jsonify(station_all)

IndentationError: unexpected indent (<ipython-input-5-4e94d9638cec>, line 3)

In [ ]:
# Query the dates and temperature observations of the most active station for the last year of data.
# Return a JSON list of Temperature Observations (tobs) for the previous year.

@app.route("/api/v1.0/tobs")
def tobs():
    prev_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)
    station_tobs = session.query(Measurement.tobs, Measurement.date).\
    filter(Measurement.station == 'USC00519281', Measurement.date >= prev_year).all()
    tobs_list = []
    for i in range(len(station_tobs)):
        station_list.append({'tobs':station_tobs[i][3],'date':station_tobs[i][1]})
    return jsonify(tobs_list)

In [ ]:
# Return a JSON list of the minimum temperature, the average temperature, and the max temperature for 
# a given start or start-end range.
# When given the start only, calculate TMIN, TAVG, and TMAX for all dates greater than and equal to the start date.
# When given the start and the end date, calculate the TMIN, TAVG, and TMAX for dates between the start and end date inclusive.

@app.route("/api/v1.0/<start>")
def temps_start(start):
    start = datetime.strptime('2016-08-23', '%Y-%m-%d').date()
    start_results = session.query(func.avg(Measurement.tobs),func.max(Measurement.tobs),func.min(Measurement.tobs).\
               filter(Measurement.date >= start)
    start_tobs_list = []   
    for i in start_results:
       dict = {}
       dict["TMIN"] = float(tobs[1])                     
       dict["TMAX"] = float(tobs[0])
       dict["TAVG"] = float(tobs[2])
       start_tobs_list.append(dict)
    return jsonify(start_tobs_list)

In [ ]:
@app.route("/api/v1.0/<start>/<end>")
def calc_temps_end(start,end):
    start = datetime.strptime('2016-08-23', '%Y-%m-%d').date()
    end = datetime.strptime('2017-08-23', '%Y-%m-%d').date()
    end_results = session.query(func.avg(Measurement.tobs),func.max(Measurement.tobs),func.min(Measurement.tobs).\
               filter(Measurement.date >= start)                     
    start_end_tobs_list = []
    for i in start_end_tobs_list:
       dict = {}
       dict["TMIN"] = float(tobs[1])                     
       dict["TMAX"] = float(tobs[0])
       dict["TAVG"] = float(tobs[2])
       start_end_tobs_list.append(dict)
    return jsonify(start__end_tobs_list)

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)